In [1]:
import numpy as np
import os
import subprocess
from fnmatch import fnmatch
import ffmpegSplit
# importlib is imported (optional) so the notebook can reload outside scripts. Without it if the script is changed, the change will not be registed in the notebook
import importlib
importlib.reload(ffmpegSplit)

from weatherGet import getWeather
from weatherDenmarkGet import getClosestStation, getWeatherDK

import pandas as pd
from datetime import datetime,timedelta
import re

# Video Manipulation Class

The class contains a number of functions for working on video files. The class calls functions from the helper class ffmpegSplit, which is based on the work from https://github.com/ToastyMallows/ffmpegSplit

All functions require the installation of ffmpeg https://ffmpeg.org/download.html on the system and use subprocesses to call it from the python code

Each of the parts of the manipulation class works on different type of data:

- **FixClips** requires the input to be large raw data video files. The files need have a specific naming convention - YYYYMMDDHHMM_YYYYMMDDHHMM, where the first one signifies the starting date and time, while the second one signifies the end date and time. The function recodes the file and removes any possible problems with segmented internal timeline or missing clip parts. The recoded videos are saved in the same subdirectory schema as the input videos 
- **splitIntoDays** takes the output directories from **FixClips** and separates the input files into single days. If the file has data larger than a single day it is split at 0:00 every day. If the file contains data less than 1 day it is renamed with the correct beginning and end time. The output from the function has the same naming convention as the previous function
- **cutIntoClips** takes the output directories from **splitIntoDays**. For each day the algorithm splits the file into 2-minute long clips for every 30 min of footage. If the input file contains data less than 2 minutes, then the clip is directly saved and renamed. The clips are renamed with the convention - clip_{number of clip}_{HHMM}, where the number of the clip is reset to 0 for each day at 0:00 or the starting time. The hour and minutes are calculated from the starting date time of the input video, the length of the clips (in our case 2 minutes) and the number of the clip
- **desaturateClip** - as the clips show thermal data, having them in YUV format, does not provide any additional information, but just makes them larger. We desaturate the clips by zeroing out the U and V components for each clip and recoding it. The input and output clips have the same naming convention and position schema in the sub-folders. 


In [2]:
class VideoManipulation():
    def __init__(self, pattern, excludeDirs = [""]):
        
        self.pattern = pattern
        self.excludeDirs = excludeDirs
        
    def desaturateClips(self, inputFileDir, saveFileDir, overwrite = False):
        for path, subdirs, files in os.walk( os.path.join(".", inputFileDir), topdown=True):
            
            for name in files:
                
                if fnmatch(name, self.pattern):
                    
                    curr_videoFilePath = os.path.join(path, name)
                    curr_saveFilePath = os.path.join(saveFileDir,os.path.normpath(path).split("\\")[-1])

                    os.makedirs(curr_saveFilePath, exist_ok=True)
                    ffmpegSplit.desaturate_video(curr_videoFilePath, curr_saveFilePath, name, overwrite)
        
    def fixClips(self, inputFileDir, saveFileDir, overwrite = False):
        for path, subdirs, files in os.walk( os.path.join(".", inputFileDir), topdown=True):

            for name in files:

                if fnmatch(name, self.pattern):

                    curr_videoFilePath = os.path.join(path, name)
                    curr_saveFilePath = os.path.join(saveFileDir,os.path.normpath(path).split("\\")[-1])

                    print(curr_videoFilePath)
                    print(curr_saveFilePath)
                    print("-----")
    
                    os.makedirs(curr_saveFilePath, exist_ok=True)
                    ffmpegSplit.fix_video(curr_videoFilePath, curr_saveFilePath, name, overwrite)
        
        
    def splitIntoDays(self,inputFileDir, outputFileDir, day_time_regex = '%Y%m%d%H%M', day_regex = '%Y%m%d', overwrite = False):
        
        clip_counter = 0

        for path, subdirs, files in os.walk( os.path.join(".", inputFileDir), topdown=True):
    
            for name in files:

                if fnmatch(name, pattern):

                    
                    curr_saveFilePath = os.path.join(outputFileDir,os.path.normpath(path).split("\\")[-1])
                    
                    os.makedirs(curr_saveFilePath, exist_ok=True)
                    clip_counter = ffmpegSplit.split_into_days(curr_saveFilePath, path, name, clip_counter, day_time_regex, day_regex, overwrite)
                    
        
        
    def cutIntoClips(self, skip, splitLength, inputFileDir, saveDir = "",overwrite = False):
        
        for path, subdirs, files in os.walk( os.path.join(".", inputFileDir), topdown=True):

            clip_counter = 0
            prev_fileName = ""
            for name in files:
                
                filename, file_extension = os.path.splitext(name)
                    
                if fnmatch(name, self.pattern):
                    
                    curr_videoFilePath = os.path.join(path, name)
                    
                    curr_dirName = filename
                    
                    
                    outputDirName_list = path.split("\\")
                    outputDirName = outputDirName_list[-1]
                    
                    outputDirName = os.path.join(saveDir, outputDirName) 
                    
                    filename_str = filename.split("_")[0]
                    
                    print(outputDirName)
                    print(filename_str)
                    print(name)
                    

                    os.makedirs(outputDirName, exist_ok=True)
                    
                    if (prev_fileName != outputDirName):
                        clip_counter = 0
                        
                    
                    print("------")
                            
                    clip_counter = ffmpegSplit.split_by_seconds(curr_videoFilePath,skip = skip,output = outputDirName, split_length = splitLength, startTime_str = filename_str, clip_counter = clip_counter, overwrite = overwrite)

                    prev_fileName = outputDirName

                    

# Weather Gather Class

The class contains functions for gathering weather information for each produced clip and combining them with the clip's folder structure and the timestamp into a metadata file.

The class also contains a natural sorting algorithm taken from the cited work. 

The weather data can be gathered from two different weather APIs:
- Meteostat - https://dev.meteostat.net/ - which contains weather data in a hourly basis
- Danish metereological Institute (DMI) - https://confluence.govcloud.dk/display/FDAPI - which contains weather data in 10 minute segments

Each of the two APIs returns slightly different weather data features:
- Meteostat - 'Temperature', 'Humidity','Precipitation', 'Dew Point', 'Wind Direction', 'Wind Speed', 'Pressure'
- DMI - 'Temperature', 'Humidity','Precipitation latest 10 min', 'Dew Point', 'Wind Direction', 'Wind Speed', 'Sun Radiation Intensity', 'Min of sunshine latest 10 min'

For the Meteostat API, a free python library is required, while the DMI API requires an additional registration and obtaining a free unique API key. The two helper functions are used **getWeather** for Meteostat and **getWeatherDK** DMI. In addition the DMI function requires an additional function for determining the closest station to the provided latitude/longitude. Both functions require a given position.

Each clip is processed by taking the day and time data and finding the closest returned weather forecast data to it. This data is saved into a *csv* file together with the timestamp and the folder structure of the clip, representing the date it was taken. If no weather data is present for the required date and time of the clip a list of NaNs is returned. If a specific data feature is not present it is set to 0. The finished csv is saved in the same outer directory as the date folders for the clips.

In [3]:
class WeatherGather():
    def __init__(self, pattern, inputDir, saveDir, weatherSystem):
        
        
        self.pattern = pattern
        self.inputDir = inputDir
        self.saveDir = saveDir
        
        self.weatherSystem = weatherSystem
        
        
    def atoi(self, text):
        return int(text) if text.isdigit() else text

    def natural_keys(self, text):
        '''
        alist.sort(key=natural_keys) sorts in human order
        http://nedbatchelder.com/blog/200712/human_sorting.html
        (See Toothy's implementation in the comments)
         '''
        return [ self.atoi(c) for c in re.split(r'(\d+)', text) ]
    
    
    def hour_rounder(self, t):
        # Rounds to nearest hour by adding a timedelta hour if minute >= 30
        return (t.replace(second=0, microsecond=0, minute=0, hour=t.hour)
                   +timedelta(hours=t.minute//30))
        
        
    def createWeatherData(self):
        results = []
        for path, subdirs, files in os.walk( os.path.join(".", self.inputDir), topdown=True):
            
            files.sort(key=self.natural_keys)
            
            pathParts = path.split("\\")
            
            if len(pathParts) == 4:
                
                print(pathParts)
                start_d_str = pathParts[-1]
                
                
                # Get last clip to calculate the end datetime for gathering data
                
                filename_last, _ = os.path.splitext(files[-1])
                name_clipTime_last_str = filename_last.split("_")
                
                filename_first, _ = os.path.splitext(files[0])
                name_clipTime_first_str = filename_first.split("_")
                
                
                
                start_d = datetime.strptime(start_d_str, '%Y%m%d')
                start_h = datetime.strptime(name_clipTime_first_str[-1], '%H%M')
                end_h = datetime.strptime(name_clipTime_last_str[-1], '%H%M')
                
                startDT = datetime.combine(start_d.date(), start_h.time())
                endDT = datetime.combine(start_d.date(), end_h.time())
                
                print(f"{path}   {startDT}    {endDT}" )

                if self.weatherSystem == "meteostat":
                    weatherData = getWeather(startDT, endDT,"hourly")
                elif self.weatherSystem == "dmi":
                    closestStation = getClosestStation()
                    weatherData = getWeatherDK(startDT,endDT,closestStation)
                    
                weatherData.fillna(0)
                
            
            for name in files:
                
                if fnmatch(name, self.pattern):
                    
                    filename, file_extension = os.path.splitext(name)
                
                    
                    name_clipTime_str = filename.split("_")
                    
                    clip_h = datetime.strptime(name_clipTime_str[-1], '%H%M')
                    date_object_clip_start = datetime.combine(start_d.date(), clip_h.time())

                    if not weatherData.empty:
                        
                        if self.weatherSystem == "meteostat":
                            currWeather = weatherData.iloc[weatherData.index.get_loc(self.hour_rounder(date_object_clip_start),method='nearest')]
                            print(f"video clip start: {date_object_clip_start}, rounded to: {self.hour_rounder(date_object_clip_start)},  temp: {currWeather['temp']}, humidity: {currWeather['rhum']}, precipitation: {currWeather['prcp']}, wind dir: {currWeather['wdir']}, wind speed: {currWeather['wspd']}, presure: {currWeather['pres']}")
                            results.append([start_d_str,filename,date_object_clip_start,currWeather['temp'],currWeather['rhum'], currWeather['prcp'],currWeather['dwpt'], currWeather['wdir'], currWeather['wspd'], currWeather['pres']])
                        elif self.weatherSystem == "dmi":
                            currWeather = weatherData.iloc[weatherData.index.get_loc(date_object_clip_start,method='nearest')]
                            # print(f"video clip start: {date_object_clip_start},  temp: {currWeather['temp_dry']}, humidity: {currWeather['humidity']}, precipitation: {currWeather['precip_past10min']},dew_point: {currWeather['temp_dew']}, wind dir: {currWeather['wind_dir']}, wind speed: {currWeather['wind_speed']}, radiation: {currWeather['radia_glob']}, sun: {currWeather['sun_last10min_glob']}")
                            results.append([start_d_str,filename,date_object_clip_start,currWeather['temp_dry'],currWeather['humidity'], currWeather['precip_past10min'],currWeather['temp_dew'], currWeather['wind_dir'], currWeather['wind_speed'], currWeather['radia_glob'], currWeather['sun_last10min_glob']])
                            
                    else:
                        print("Weather DF empty")
                        if self.weatherSystem == "meteostat":
                            results.append([start_d_str,filename,date_object_clip_start,np.NaN,np.NaN, np.NaN,np.NaN,np.NaN, np.NaN, np.NaN])
                        elif self.weatherSystem == "dmi":
                            results.append([start_d_str,filename,date_object_clip_start,np.NaN,np.NaN, np.NaN,np.NaN,np.NaN, np.NaN, np.NaN, np.NaN])
                    
                   
        df = pd.DataFrame(results)
        if self.weatherSystem == "meteostat":
            
            df.columns =['Folder name','Clip Name', 'DateTime', 'Temperature', 'Humidity','Precipitation', 'Dew Point', 'Wind Direction', 'Wind Speed', 'Pressure']
        elif self.weatherSystem == "dmi":
            df.columns =['Folder name','Clip Name', 'DateTime', 'Temperature', 'Humidity','Precipitation latest 10 min', 'Dew Point', 'Wind Direction', 'Wind Speed', 'Sun Radiation Intensity', 'Min of sunshine latest 10 min']
        
        df.to_csv(self.saveDir,index=False)

# Initialize the video manipulator class

In [4]:
print(f"Current working directory: {os.getcwd()}")
pattern = "*.mp4"

vidManipulator = VideoManipulation(pattern)

dir_jinsong_files = r"raw_files_directory"
dir_processed_files = r"main_output_directory"

input_paths = [f.path for f in os.scandir(dir_jinsong_files) if f.is_dir()]
print(input_paths)




Current working directory: D:\2021_workstuff\VideoManipulation
['E:\\0_Jinsong_files\\5_April']


# Get input video files and recode them to remove fragmentation

In [5]:

for path in input_paths:
    print(path)
    

    input_video_dir = path

    output_fixed_video_dir = os.path.join(dir_processed_files, "inputs_fixed")
    
    vidManipulator.fixClips(input_video_dir,output_fixed_video_dir)



E:\0_Jinsong_files\5_April
E:\0_Jinsong_files\5_April\202103311957_202104010131.mp4
E:\0_Monthly_videoPipeline\inputs_fixed\5_April
-----
202103311957_202104010131
About to run: ffmpeg -y -i E:\0_Jinsong_files\5_April\202103311957_202104010131.mp4 -c:v libx264 E:\0_Monthly_videoPipeline\inputs_fixed\5_April\202103311957_202104010131.mp4
E:\0_Jinsong_files\5_April\202104010131_202104010900.mp4
E:\0_Monthly_videoPipeline\inputs_fixed\5_April
-----
202104010131_202104010900
About to run: ffmpeg -y -i E:\0_Jinsong_files\5_April\202104010131_202104010900.mp4 -c:v libx264 E:\0_Monthly_videoPipeline\inputs_fixed\5_April\202104010131_202104010900.mp4
E:\0_Jinsong_files\5_April\202104010900_202104011305.mp4
E:\0_Monthly_videoPipeline\inputs_fixed\5_April
-----
202104010900_202104011305
About to run: ffmpeg -y -i E:\0_Jinsong_files\5_April\202104010900_202104011305.mp4 -c:v libx264 E:\0_Monthly_videoPipeline\inputs_fixed\5_April\202104010900_202104011305.mp4
E:\0_Jinsong_files\5_April\2021040113

E:\0_Jinsong_files\5_April\202104070358_202104071018.mp4
E:\0_Monthly_videoPipeline\inputs_fixed\5_April
-----
202104070358_202104071018
About to run: ffmpeg -y -i E:\0_Jinsong_files\5_April\202104070358_202104071018.mp4 -c:v libx264 E:\0_Monthly_videoPipeline\inputs_fixed\5_April\202104070358_202104071018.mp4
E:\0_Jinsong_files\5_April\202104071018_202104071715.mp4
E:\0_Monthly_videoPipeline\inputs_fixed\5_April
-----
202104071018_202104071715
About to run: ffmpeg -y -i E:\0_Jinsong_files\5_April\202104071018_202104071715.mp4 -c:v libx264 E:\0_Monthly_videoPipeline\inputs_fixed\5_April\202104071018_202104071715.mp4
E:\0_Jinsong_files\5_April\202104071715_202104071757.mp4
E:\0_Monthly_videoPipeline\inputs_fixed\5_April
-----
202104071715_202104071757
About to run: ffmpeg -y -i E:\0_Jinsong_files\5_April\202104071715_202104071757.mp4 -c:v libx264 E:\0_Monthly_videoPipeline\inputs_fixed\5_April\202104071715_202104071757.mp4
E:\0_Jinsong_files\5_April\202104071759_202104072357.mp4
E:\0_Mo

E:\0_Jinsong_files\5_April\202104130226_202104130914.mp4
E:\0_Monthly_videoPipeline\inputs_fixed\5_April
-----
202104130226_202104130914
About to run: ffmpeg -y -i E:\0_Jinsong_files\5_April\202104130226_202104130914.mp4 -c:v libx264 E:\0_Monthly_videoPipeline\inputs_fixed\5_April\202104130226_202104130914.mp4
E:\0_Jinsong_files\5_April\202104130914_202104131421.mp4
E:\0_Monthly_videoPipeline\inputs_fixed\5_April
-----
202104130914_202104131421
About to run: ffmpeg -y -i E:\0_Jinsong_files\5_April\202104130914_202104131421.mp4 -c:v libx264 E:\0_Monthly_videoPipeline\inputs_fixed\5_April\202104130914_202104131421.mp4
E:\0_Jinsong_files\5_April\202104131421_202104131748.mp4
E:\0_Monthly_videoPipeline\inputs_fixed\5_April
-----
202104131421_202104131748
About to run: ffmpeg -y -i E:\0_Jinsong_files\5_April\202104131421_202104131748.mp4 -c:v libx264 E:\0_Monthly_videoPipeline\inputs_fixed\5_April\202104131421_202104131748.mp4
E:\0_Jinsong_files\5_April\202104131748_202104132255.mp4
E:\0_Mo

E:\0_Jinsong_files\5_April\202104181954_202104182143.mp4
E:\0_Monthly_videoPipeline\inputs_fixed\5_April
-----
202104181954_202104182143
About to run: ffmpeg -y -i E:\0_Jinsong_files\5_April\202104181954_202104182143.mp4 -c:v libx264 E:\0_Monthly_videoPipeline\inputs_fixed\5_April\202104181954_202104182143.mp4
E:\0_Jinsong_files\5_April\202104182145_202104190458.mp4
E:\0_Monthly_videoPipeline\inputs_fixed\5_April
-----
202104182145_202104190458
About to run: ffmpeg -y -i E:\0_Jinsong_files\5_April\202104182145_202104190458.mp4 -c:v libx264 E:\0_Monthly_videoPipeline\inputs_fixed\5_April\202104182145_202104190458.mp4
E:\0_Jinsong_files\5_April\202104190458_202104190921.mp4
E:\0_Monthly_videoPipeline\inputs_fixed\5_April
-----
202104190458_202104190921
About to run: ffmpeg -y -i E:\0_Jinsong_files\5_April\202104190458_202104190921.mp4 -c:v libx264 E:\0_Monthly_videoPipeline\inputs_fixed\5_April\202104190458_202104190921.mp4
E:\0_Jinsong_files\5_April\202104190921_202104191338.mp4
E:\0_Mo

E:\0_Jinsong_files\5_April\202104231738_202104232130.mp4
E:\0_Monthly_videoPipeline\inputs_fixed\5_April
-----
202104231738_202104232130
About to run: ffmpeg -y -i E:\0_Jinsong_files\5_April\202104231738_202104232130.mp4 -c:v libx264 E:\0_Monthly_videoPipeline\inputs_fixed\5_April\202104231738_202104232130.mp4
E:\0_Jinsong_files\5_April\202104232130_202104232307.mp4
E:\0_Monthly_videoPipeline\inputs_fixed\5_April
-----
202104232130_202104232307
About to run: ffmpeg -y -i E:\0_Jinsong_files\5_April\202104232130_202104232307.mp4 -c:v libx264 E:\0_Monthly_videoPipeline\inputs_fixed\5_April\202104232130_202104232307.mp4
E:\0_Jinsong_files\5_April\202104232308_202104240535.mp4
E:\0_Monthly_videoPipeline\inputs_fixed\5_April
-----
202104232308_202104240535
About to run: ffmpeg -y -i E:\0_Jinsong_files\5_April\202104232308_202104240535.mp4 -c:v libx264 E:\0_Monthly_videoPipeline\inputs_fixed\5_April\202104232308_202104240535.mp4
E:\0_Jinsong_files\5_April\202104240535_202104241018.mp4
E:\0_Mo

E:\0_Jinsong_files\5_April\202104290339_202104290933.mp4
E:\0_Monthly_videoPipeline\inputs_fixed\5_April
-----
202104290339_202104290933
About to run: ffmpeg -y -i E:\0_Jinsong_files\5_April\202104290339_202104290933.mp4 -c:v libx264 E:\0_Monthly_videoPipeline\inputs_fixed\5_April\202104290339_202104290933.mp4
E:\0_Jinsong_files\5_April\202104290933_202104291156.mp4
E:\0_Monthly_videoPipeline\inputs_fixed\5_April
-----
202104290933_202104291156
About to run: ffmpeg -y -i E:\0_Jinsong_files\5_April\202104290933_202104291156.mp4 -c:v libx264 E:\0_Monthly_videoPipeline\inputs_fixed\5_April\202104290933_202104291156.mp4
E:\0_Jinsong_files\5_April\202104291156_202104291510.mp4
E:\0_Monthly_videoPipeline\inputs_fixed\5_April
-----
202104291156_202104291510
About to run: ffmpeg -y -i E:\0_Jinsong_files\5_April\202104291156_202104291510.mp4 -c:v libx264 E:\0_Monthly_videoPipeline\inputs_fixed\5_April\202104291156_202104291510.mp4
E:\0_Jinsong_files\5_April\202104291510_202104291851.mp4
E:\0_Mo

# Get recoded video files and split them into days. Put them in separate directories

In [6]:
input_fixed_video_dir = output_fixed_video_dir
output_day_segmented_video_dir = os.path.join(dir_processed_files, "inputs_fixed_daily") 

if not os.path.exists(output_day_segmented_video_dir):
    os.makedirs(output_day_segmented_video_dir, exist_ok=True)

vidManipulator.splitIntoDays(input_fixed_video_dir, output_day_segmented_video_dir)


Video length in seconds: 8622
202101081158
No overwrite and file exists
202101081158_202101081423.mp4 just copy
Video length in seconds: 46004
202101081627
 202101081627_202101090515.mp4 needs split
File E:\0_Monthly_videoPipeline\inputs_fixed_daily\1_January\20210108\202101081627_202101082359.mp4 exists and overwrite is not set to true 
File E:\0_Monthly_videoPipeline\inputs_fixed_daily\1_January\20210109\202101090000_202101090513.mp4 exists and overwrite is not set to true 
Video length in seconds: 33133
202101090515
No overwrite and file exists
202101090515_202101091524.mp4 just copy
Video length in seconds: 28046
202101091524
No overwrite and file exists
202101091524_202101092311.mp4 just copy
Video length in seconds: 6760
202101092311
 202101092311_202101100106.mp4 needs split
File E:\0_Monthly_videoPipeline\inputs_fixed_daily\1_January\20210109\202101092311_202101092359.mp4 exists and overwrite is not set to true 
File E:\0_Monthly_videoPipeline\inputs_fixed_daily\1_January\20210

Video length in seconds: 22444
202101230347
No overwrite and file exists
202101230347_202101231108.mp4 just copy
Video length in seconds: 18994
202101231108
No overwrite and file exists
202101231108_202101231624.mp4 just copy
Video length in seconds: 30027
202101231625
 202101231625_202101240045.mp4 needs split
File E:\0_Monthly_videoPipeline\inputs_fixed_daily\1_January\20210123\202101231625_202101232359.mp4 exists and overwrite is not set to true 
File E:\0_Monthly_videoPipeline\inputs_fixed_daily\1_January\20210124\202101240000_202101240045.mp4 exists and overwrite is not set to true 
Video length in seconds: 37042
202101240045
No overwrite and file exists
202101240045_202101241113.mp4 just copy
Video length in seconds: 1971
202101241113
No overwrite and file exists
202101241113_202101241249.mp4 just copy
Video length in seconds: 38272
202101241249
No overwrite and file exists
202101241249_202101242327.mp4 just copy
Video length in seconds: 37560
202101242327
 202101242327_202101251

No overwrite and file exists
202102060309_202102060626.mp4 just copy
Video length in seconds: 8869
202102060626
No overwrite and file exists
202102060626_202102060857.mp4 just copy
Video length in seconds: 13935
202102061000
No overwrite and file exists
202102061000_202102061352.mp4 just copy
Video length in seconds: 11868
202102061352
No overwrite and file exists
202102061352_202102061710.mp4 just copy
Video length in seconds: 20057
202102061710
No overwrite and file exists
202102061710_202102062244.mp4 just copy
Video length in seconds: 14167
202102062244
 202102062244_202102070241.mp4 needs split
File E:\0_Monthly_videoPipeline\inputs_fixed_daily\2_February\20210206\202102062244_202102062359.mp4 exists and overwrite is not set to true 
File E:\0_Monthly_videoPipeline\inputs_fixed_daily\2_February\20210207\202102070000_202102070240.mp4 exists and overwrite is not set to true 
Video length in seconds: 13087
202102070241
No overwrite and file exists
202102070241_202102070806.mp4 just c

Video length in seconds: 31945
202102210303
No overwrite and file exists
202102210303_202102211157.mp4 just copy
Video length in seconds: 20589
202102211157
No overwrite and file exists
202102211157_202102211740.mp4 just copy
Video length in seconds: 35170
202102211741
 202102211741_202102220329.mp4 needs split
File E:\0_Monthly_videoPipeline\inputs_fixed_daily\2_February\20210221\202102211741_202102212359.mp4 exists and overwrite is not set to true 
File E:\0_Monthly_videoPipeline\inputs_fixed_daily\2_February\20210222\202102220000_202102220327.mp4 exists and overwrite is not set to true 
Video length in seconds: 28360
202102220329
No overwrite and file exists
202102220329_202102221245.mp4 just copy
Video length in seconds: 17192
202102221245
No overwrite and file exists
202102221245_202102221731.mp4 just copy
Video length in seconds: 38235
202102221731
 202102221731_202102230822.mp4 needs split
File E:\0_Monthly_videoPipeline\inputs_fixed_daily\2_February\20210222\202102221731_202102

Video length in seconds: 21802
202005200254
No overwrite and file exists
202005200254_20200520‏‎0858.mp4 just copy
Video length in seconds: 16614
202005201053
No overwrite and file exists
202005201053_202005201532.mp4 just copy
Video length in seconds: 11886
202005201532
No overwrite and file exists
202005201532_202005201850.mp4 just copy
Video length in seconds: 15772
202005201850
No overwrite and file exists
202005201850_202005202313.mp4 just copy
Video length in seconds: 4292
202005202313
 202005202313_202005210024.mp4 needs split
File E:\0_Monthly_videoPipeline\inputs_fixed_daily\3_SummerData\20200520\202005202313_202005202359.mp4 exists and overwrite is not set to true 
File E:\0_Monthly_videoPipeline\inputs_fixed_daily\3_SummerData\20200521\202005210000_202005210024.mp4 exists and overwrite is not set to true 
Video length in seconds: 6938
202005200858
No overwrite and file exists
20200520‏‎0858_202005201053.mp4 just copy
Video length in seconds: 31638
202005210024
No overwrite a

...

Video length in seconds: 23807
202103071601
No overwrite and file exists
202103071601_202103072238.mp4 just copy
Video length in seconds: 28651
202103072238
 202103072238_202103081115.mp4 needs split
File E:\0_Monthly_videoPipeline\inputs_fixed_daily\4_March\20210307\202103072238_202103072359.mp4 exists and overwrite is not set to true 
File E:\0_Monthly_videoPipeline\inputs_fixed_daily\4_March\20210308\202103080000_202103080635.mp4 exists and overwrite is not set to true 
Video length in seconds: 18569
202103081115
No overwrite and file exists
202103081115_202103081638.mp4 just copy
Video length in seconds: 12304
202103081638
No overwrite and file exists
202103081638_202103082028.mp4 just copy
Video length in seconds: 33182
202103082034
 202103082034_202103090547.mp4 needs split
File E:\0_Monthly_videoPipeline\inputs_fixed_daily\4_March\20210308\202103082034_202103082359.mp4 exists and overwrite is not set to true 
File E:\0_Monthly_videoPipeline\inputs_fixed_daily\4_March\20210309\20

Video length in seconds: 20741
202103210635
No overwrite and file exists
202103210635_202103211221.mp4 just copy
Video length in seconds: 13951
202103211221
No overwrite and file exists
202103211221_202103211658.mp4 just copy
Video length in seconds: 20718
202103211658
No overwrite and file exists
202103211658_202103212243.mp4 just copy
Video length in seconds: 32873
202103212243
 202103212243_202103220752.mp4 needs split
File E:\0_Monthly_videoPipeline\inputs_fixed_daily\4_March\20210321\202103212243_202103212359.mp4 exists and overwrite is not set to true 
File E:\0_Monthly_videoPipeline\inputs_fixed_daily\4_March\20210322\202103220000_202103220750.mp4 exists and overwrite is not set to true 
Video length in seconds: 21444
202103220752
No overwrite and file exists
202103220752_202103221349.mp4 just copy
Video length in seconds: 242
202103221349
No overwrite and file exists
202103221349_202103221419.mp4 just copy
Video length in seconds: 19181
202103221419
No overwrite and file exists

Video length in seconds: 18510
202104020936
202104020936_202104021445.mp4 just copy
Video length in seconds: 10964
202104021445
202104021445_202104021748.mp4 just copy
Video length in seconds: 17860
202104021748
202104021748_202104022245.mp4 just copy
Video length in seconds: 8807
202104022245
 202104022245_202104030112.mp4 needs split
About to run: ffmpeg -y -ss 0 -i E:\0_Monthly_videoPipeline\inputs_fixed\5_April\202104022245_202104030112.mp4 -to 4499.0 -c copy E:\0_Monthly_videoPipeline\inputs_fixed_daily\5_April\20210402\202104022245_202104022359.mp4
About to run: ffmpeg -y -ss 4499.0 -i E:\0_Monthly_videoPipeline\inputs_fixed\5_April\202104022245_202104030112.mp4 -to 8807 -c copy E:\0_Monthly_videoPipeline\inputs_fixed_daily\5_April\20210403\202104030000_202104030111.mp4
Video length in seconds: 20593
202104030112
202104030112_202104030655.mp4 just copy
Video length in seconds: 19568
202104030655
202104030655_202104031222.mp4 just copy
Video length in seconds: 4746
202104031222
20

202104121750_202104121901.mp4 just copy
Video length in seconds: 6214
202104121903
202104121903_202104122048.mp4 just copy
Video length in seconds: 19247
202104122105
 202104122105_202104130226.mp4 needs split
About to run: ffmpeg -y -ss 0 -i E:\0_Monthly_videoPipeline\inputs_fixed\5_April\202104122105_202104130226.mp4 -to 10499.0 -c copy E:\0_Monthly_videoPipeline\inputs_fixed_daily\5_April\20210412\202104122105_202104122359.mp4
About to run: ffmpeg -y -ss 10499.0 -i E:\0_Monthly_videoPipeline\inputs_fixed\5_April\202104122105_202104130226.mp4 -to 19247 -c copy E:\0_Monthly_videoPipeline\inputs_fixed_daily\5_April\20210413\202104130000_202104130225.mp4
Video length in seconds: 24459
202104130226
202104130226_202104130914.mp4 just copy
Video length in seconds: 18406
202104130914
202104130914_202104131421.mp4 just copy
Video length in seconds: 12408
202104131421
202104131421_202104131748.mp4 just copy
Video length in seconds: 18415
202104131748
202104131748_202104132255.mp4 just copy
Vi

About to run: ffmpeg -y -ss 1079.0 -i E:\0_Monthly_videoPipeline\inputs_fixed\5_April\202104212342_202104220404.mp4 -to 15720 -c copy E:\0_Monthly_videoPipeline\inputs_fixed_daily\5_April\20210422\202104220000_202104220404.mp4
Video length in seconds: 14893
202104220404
202104220404_202104220813.mp4 just copy
Video length in seconds: 13735
202104220813
202104220813_202104221202.mp4 just copy
Video length in seconds: 12280
202104221202
202104221202_202104221526.mp4 just copy
Video length in seconds: 11096
202104221526
202104221526_202104221831.mp4 just copy
Video length in seconds: 15653
202104221831
202104221831_202104222252.mp4 just copy
Video length in seconds: 8129
202104222252
 202104222252_202104230108.mp4 needs split
About to run: ffmpeg -y -ss 0 -i E:\0_Monthly_videoPipeline\inputs_fixed\5_April\202104222252_202104230108.mp4 -to 4079.0 -c copy E:\0_Monthly_videoPipeline\inputs_fixed_daily\5_April\20210422\202104222252_202104222359.mp4
About to run: ffmpeg -y -ss 4079.0 -i E:\0_M

# Get the daily split video files and extract clip segments from them

In [8]:
input_day_segmented_video_dir = output_day_segmented_video_dir
output_segmented_clips_dir = os.path.join(dir_processed_files, "output_clips") 

skip = 14
splitLength = 120
vidManipulator.cutIntoClips(skip, splitLength, input_day_segmented_video_dir, output_segmented_clips_dir)

E:\0_Monthly_videoPipeline\output_clips\20210108
202101081158
202101081158_202101081421.mp4
------
Video length in seconds: 8622
E:\0_Monthly_videoPipeline\inputs_fixed_daily\1_January\20210108\202101081158_202101081421
File E:\0_Monthly_videoPipeline\output_clips\20210108\clip_0_1158.mp4 exists and overwrite is not set to true 
File E:\0_Monthly_videoPipeline\output_clips\20210108\clip_1_1226.mp4 exists and overwrite is not set to true 
File E:\0_Monthly_videoPipeline\output_clips\20210108\clip_2_1254.mp4 exists and overwrite is not set to true 
File E:\0_Monthly_videoPipeline\output_clips\20210108\clip_3_1322.mp4 exists and overwrite is not set to true 
File E:\0_Monthly_videoPipeline\output_clips\20210108\clip_4_1350.mp4 exists and overwrite is not set to true 
File E:\0_Monthly_videoPipeline\output_clips\20210108\clip_5_1418.mp4 exists and overwrite is not set to true 
E:\0_Monthly_videoPipeline\output_clips\20210108
202101081627
202101081627_202101082359.mp4
------
Video length in

Video length in seconds: 3826
E:\0_Monthly_videoPipeline\inputs_fixed_daily\1_January\20210110\202101100000_202101100103
File E:\0_Monthly_videoPipeline\output_clips\20210110\clip_0_0000.mp4 exists and overwrite is not set to true 
File E:\0_Monthly_videoPipeline\output_clips\20210110\clip_1_0028.mp4 exists and overwrite is not set to true 
File E:\0_Monthly_videoPipeline\output_clips\20210110\clip_2_0056.mp4 exists and overwrite is not set to true 
E:\0_Monthly_videoPipeline\output_clips\20210110
202101100106
202101100106_202101101034.mp4
------
Video length in seconds: 34129
E:\0_Monthly_videoPipeline\inputs_fixed_daily\1_January\20210110\202101100106_202101101034
File E:\0_Monthly_videoPipeline\output_clips\20210110\clip_3_0106.mp4 exists and overwrite is not set to true 
File E:\0_Monthly_videoPipeline\output_clips\20210110\clip_4_0134.mp4 exists and overwrite is not set to true 
File E:\0_Monthly_videoPipeline\output_clips\20210110\clip_5_0202.mp4 exists and overwrite is not set t

Video length in seconds: 26519
E:\0_Monthly_videoPipeline\inputs_fixed_daily\1_January\20210111\202101111638_202101112359
File E:\0_Monthly_videoPipeline\output_clips\20210111\clip_36_1638.mp4 exists and overwrite is not set to true 
File E:\0_Monthly_videoPipeline\output_clips\20210111\clip_37_1706.mp4 exists and overwrite is not set to true 
File E:\0_Monthly_videoPipeline\output_clips\20210111\clip_38_1734.mp4 exists and overwrite is not set to true 
File E:\0_Monthly_videoPipeline\output_clips\20210111\clip_39_1802.mp4 exists and overwrite is not set to true 
File E:\0_Monthly_videoPipeline\output_clips\20210111\clip_40_1830.mp4 exists and overwrite is not set to true 
File E:\0_Monthly_videoPipeline\output_clips\20210111\clip_41_1858.mp4 exists and overwrite is not set to true 
File E:\0_Monthly_videoPipeline\output_clips\20210111\clip_42_1926.mp4 exists and overwrite is not set to true 
File E:\0_Monthly_videoPipeline\output_clips\20210111\clip_43_1954.mp4 exists and overwrite is

File E:\0_Monthly_videoPipeline\output_clips\20210113\clip_5_0205.mp4 exists and overwrite is not set to true 
File E:\0_Monthly_videoPipeline\output_clips\20210113\clip_6_0233.mp4 exists and overwrite is not set to true 
File E:\0_Monthly_videoPipeline\output_clips\20210113\clip_7_0301.mp4 exists and overwrite is not set to true 
File E:\0_Monthly_videoPipeline\output_clips\20210113\clip_8_0329.mp4 exists and overwrite is not set to true 
File E:\0_Monthly_videoPipeline\output_clips\20210113\clip_9_0357.mp4 exists and overwrite is not set to true 
File E:\0_Monthly_videoPipeline\output_clips\20210113\clip_10_0425.mp4 exists and overwrite is not set to true 
File E:\0_Monthly_videoPipeline\output_clips\20210113\clip_11_0453.mp4 exists and overwrite is not set to true 
File E:\0_Monthly_videoPipeline\output_clips\20210113\clip_12_0521.mp4 exists and overwrite is not set to true 
File E:\0_Monthly_videoPipeline\output_clips\20210113\clip_13_0549.mp4 exists and overwrite is not set to tru

Video length in seconds: 28094
E:\0_Monthly_videoPipeline\inputs_fixed_daily\1_January\20210115\202101150029_202101150817
File E:\0_Monthly_videoPipeline\output_clips\20210115\clip_2_0029.mp4 exists and overwrite is not set to true 
File E:\0_Monthly_videoPipeline\output_clips\20210115\clip_3_0057.mp4 exists and overwrite is not set to true 
File E:\0_Monthly_videoPipeline\output_clips\20210115\clip_4_0125.mp4 exists and overwrite is not set to true 
File E:\0_Monthly_videoPipeline\output_clips\20210115\clip_5_0153.mp4 exists and overwrite is not set to true 
File E:\0_Monthly_videoPipeline\output_clips\20210115\clip_6_0221.mp4 exists and overwrite is not set to true 
File E:\0_Monthly_videoPipeline\output_clips\20210115\clip_7_0249.mp4 exists and overwrite is not set to true 
File E:\0_Monthly_videoPipeline\output_clips\20210115\clip_8_0317.mp4 exists and overwrite is not set to true 
File E:\0_Monthly_videoPipeline\output_clips\20210115\clip_9_0345.mp4 exists and overwrite is not set

...

About to run: ffmpeg -y -ss 3360 -i E:\0_Monthly_videoPipeline\inputs_fixed_daily\5_April\20210428\202104280352_202104281109.mp4 -to 120 -c copy E:\0_Monthly_videoPipeline\output_clips\20210428\clip_11_0448.mp4
About to run: ffmpeg -y -ss 5040 -i E:\0_Monthly_videoPipeline\inputs_fixed_daily\5_April\20210428\202104280352_202104281109.mp4 -to 120 -c copy E:\0_Monthly_videoPipeline\output_clips\20210428\clip_12_0516.mp4
About to run: ffmpeg -y -ss 6720 -i E:\0_Monthly_videoPipeline\inputs_fixed_daily\5_April\20210428\202104280352_202104281109.mp4 -to 120 -c copy E:\0_Monthly_videoPipeline\output_clips\20210428\clip_13_0544.mp4
About to run: ffmpeg -y -ss 8400 -i E:\0_Monthly_videoPipeline\inputs_fixed_daily\5_April\20210428\202104280352_202104281109.mp4 -to 120 -c copy E:\0_Monthly_videoPipeline\output_clips\20210428\clip_14_0612.mp4
About to run: ffmpeg -y -ss 10080 -i E:\0_Monthly_videoPipeline\inputs_fixed_daily\5_April\20210428\202104280352_202104281109.mp4 -to 120 -c copy E:\0_Month

About to run: ffmpeg -y -ss 3360 -i E:\0_Monthly_videoPipeline\inputs_fixed_daily\5_April\20210428\202104282041_202104282359.mp4 -to 120 -c copy E:\0_Monthly_videoPipeline\output_clips\20210428\clip_47_2137.mp4
About to run: ffmpeg -y -ss 5040 -i E:\0_Monthly_videoPipeline\inputs_fixed_daily\5_April\20210428\202104282041_202104282359.mp4 -to 120 -c copy E:\0_Monthly_videoPipeline\output_clips\20210428\clip_48_2205.mp4
About to run: ffmpeg -y -ss 6720 -i E:\0_Monthly_videoPipeline\inputs_fixed_daily\5_April\20210428\202104282041_202104282359.mp4 -to 120 -c copy E:\0_Monthly_videoPipeline\output_clips\20210428\clip_49_2233.mp4
About to run: ffmpeg -y -ss 8400 -i E:\0_Monthly_videoPipeline\inputs_fixed_daily\5_April\20210428\202104282041_202104282359.mp4 -to 120 -c copy E:\0_Monthly_videoPipeline\output_clips\20210428\clip_50_2301.mp4
About to run: ffmpeg -y -ss 10080 -i E:\0_Monthly_videoPipeline\inputs_fixed_daily\5_April\20210428\202104282041_202104282359.mp4 -to 120 -c copy E:\0_Month

About to run: ffmpeg -y -ss 3360 -i E:\0_Monthly_videoPipeline\inputs_fixed_daily\5_April\20210429\202104291156_202104291510.mp4 -to 120 -c copy E:\0_Monthly_videoPipeline\output_clips\20210429\clip_29_1252.mp4
About to run: ffmpeg -y -ss 5040 -i E:\0_Monthly_videoPipeline\inputs_fixed_daily\5_April\20210429\202104291156_202104291510.mp4 -to 120 -c copy E:\0_Monthly_videoPipeline\output_clips\20210429\clip_30_1320.mp4
About to run: ffmpeg -y -ss 6720 -i E:\0_Monthly_videoPipeline\inputs_fixed_daily\5_April\20210429\202104291156_202104291510.mp4 -to 120 -c copy E:\0_Monthly_videoPipeline\output_clips\20210429\clip_31_1348.mp4
About to run: ffmpeg -y -ss 8400 -i E:\0_Monthly_videoPipeline\inputs_fixed_daily\5_April\20210429\202104291156_202104291510.mp4 -to 120 -c copy E:\0_Monthly_videoPipeline\output_clips\20210429\clip_32_1416.mp4
About to run: ffmpeg -y -ss 10080 -i E:\0_Monthly_videoPipeline\inputs_fixed_daily\5_April\20210429\202104291156_202104291510.mp4 -to 120 -c copy E:\0_Month

About to run: ffmpeg -y -ss 16800 -i E:\0_Monthly_videoPipeline\inputs_fixed_daily\5_April\20210430\202104300000_202104300633.mp4 -to 120 -c copy E:\0_Monthly_videoPipeline\output_clips\20210430\clip_10_0440.mp4
About to run: ffmpeg -y -ss 18480 -i E:\0_Monthly_videoPipeline\inputs_fixed_daily\5_April\20210430\202104300000_202104300633.mp4 -to 120 -c copy E:\0_Monthly_videoPipeline\output_clips\20210430\clip_11_0508.mp4
About to run: ffmpeg -y -ss 20160 -i E:\0_Monthly_videoPipeline\inputs_fixed_daily\5_April\20210430\202104300000_202104300633.mp4 -to 120 -c copy E:\0_Monthly_videoPipeline\output_clips\20210430\clip_12_0536.mp4
About to run: ffmpeg -y -ss 21840 -i E:\0_Monthly_videoPipeline\inputs_fixed_daily\5_April\20210430\202104300000_202104300633.mp4 -to 120 -c copy E:\0_Monthly_videoPipeline\output_clips\20210430\clip_13_0604.mp4
About to run: ffmpeg -y -ss 23520 -i E:\0_Monthly_videoPipeline\inputs_fixed_daily\5_April\20210430\202104300000_202104300633.mp4 -to 120 -c copy E:\0_M

About to run: ffmpeg -y -ss 1680 -i E:\0_Monthly_videoPipeline\inputs_fixed_daily\5_April\20210430\202104302002_202104302359.mp4 -to 120 -c copy E:\0_Monthly_videoPipeline\output_clips\20210430\clip_46_2030.mp4
About to run: ffmpeg -y -ss 3360 -i E:\0_Monthly_videoPipeline\inputs_fixed_daily\5_April\20210430\202104302002_202104302359.mp4 -to 120 -c copy E:\0_Monthly_videoPipeline\output_clips\20210430\clip_47_2058.mp4
About to run: ffmpeg -y -ss 5040 -i E:\0_Monthly_videoPipeline\inputs_fixed_daily\5_April\20210430\202104302002_202104302359.mp4 -to 120 -c copy E:\0_Monthly_videoPipeline\output_clips\20210430\clip_48_2126.mp4
About to run: ffmpeg -y -ss 6720 -i E:\0_Monthly_videoPipeline\inputs_fixed_daily\5_April\20210430\202104302002_202104302359.mp4 -to 120 -c copy E:\0_Monthly_videoPipeline\output_clips\20210430\clip_49_2154.mp4
About to run: ffmpeg -y -ss 8400 -i E:\0_Monthly_videoPipeline\inputs_fixed_daily\5_April\20210430\202104302002_202104302359.mp4 -to 120 -c copy E:\0_Monthl

# Get the clip segments and remove color information from them leaving only a grayscale

In [9]:
input_segmented_clips_dir = output_segmented_clips_dir
output_grayscaled_clips_dir = os.path.join(dir_processed_files, "output_desat_clips")  

vidManipulator.desaturateClips(input_segmented_clips_dir, output_grayscaled_clips_dir)

clip_0_1331
File E:\0_Monthly_videoPipeline\output_desat_clips\20200514\clip_0_1331 exists and overwrite is not set to true 
clip_10_1744
File E:\0_Monthly_videoPipeline\output_desat_clips\20200514\clip_10_1744 exists and overwrite is not set to true 
clip_11_1812
File E:\0_Monthly_videoPipeline\output_desat_clips\20200514\clip_11_1812 exists and overwrite is not set to true 
clip_12_1840
File E:\0_Monthly_videoPipeline\output_desat_clips\20200514\clip_12_1840 exists and overwrite is not set to true 
clip_13_1908
File E:\0_Monthly_videoPipeline\output_desat_clips\20200514\clip_13_1908 exists and overwrite is not set to true 
clip_14_1936
File E:\0_Monthly_videoPipeline\output_desat_clips\20200514\clip_14_1936 exists and overwrite is not set to true 
clip_15_2004
File E:\0_Monthly_videoPipeline\output_desat_clips\20200514\clip_15_2004 exists and overwrite is not set to true 
clip_16_2032
File E:\0_Monthly_videoPipeline\output_desat_clips\20200514\clip_16_2032 exists and overwrite is not

clip_38_1620
File E:\0_Monthly_videoPipeline\output_desat_clips\20200522\clip_38_1620 exists and overwrite is not set to true 
clip_39_1648
File E:\0_Monthly_videoPipeline\output_desat_clips\20200522\clip_39_1648 exists and overwrite is not set to true 
clip_3_0124
File E:\0_Monthly_videoPipeline\output_desat_clips\20200522\clip_3_0124 exists and overwrite is not set to true 
clip_40_1716
File E:\0_Monthly_videoPipeline\output_desat_clips\20200522\clip_40_1716 exists and overwrite is not set to true 
clip_41_1744
File E:\0_Monthly_videoPipeline\output_desat_clips\20200522\clip_41_1744 exists and overwrite is not set to true 
clip_42_1812
File E:\0_Monthly_videoPipeline\output_desat_clips\20200522\clip_42_1812 exists and overwrite is not set to true 
clip_43_1840
File E:\0_Monthly_videoPipeline\output_desat_clips\20200522\clip_43_1840 exists and overwrite is not set to true 
clip_44_1908
File E:\0_Monthly_videoPipeline\output_desat_clips\20200522\clip_44_1908 exists and overwrite is not

File E:\0_Monthly_videoPipeline\output_desat_clips\20200531\clip_10_0432 exists and overwrite is not set to true 
clip_11_0500
File E:\0_Monthly_videoPipeline\output_desat_clips\20200531\clip_11_0500 exists and overwrite is not set to true 
clip_12_0528
File E:\0_Monthly_videoPipeline\output_desat_clips\20200531\clip_12_0528 exists and overwrite is not set to true 
clip_13_0556
File E:\0_Monthly_videoPipeline\output_desat_clips\20200531\clip_13_0556 exists and overwrite is not set to true 
clip_14_0624
File E:\0_Monthly_videoPipeline\output_desat_clips\20200531\clip_14_0624 exists and overwrite is not set to true 
clip_15_0652
File E:\0_Monthly_videoPipeline\output_desat_clips\20200531\clip_15_0652 exists and overwrite is not set to true 
clip_16_0720
File E:\0_Monthly_videoPipeline\output_desat_clips\20200531\clip_16_0720 exists and overwrite is not set to true 
clip_17_0748
File E:\0_Monthly_videoPipeline\output_desat_clips\20200531\clip_17_0748 exists and overwrite is not set to tru

clip_48_2153
File E:\0_Monthly_videoPipeline\output_desat_clips\20200608\clip_48_2153 exists and overwrite is not set to true 
clip_49_2221
File E:\0_Monthly_videoPipeline\output_desat_clips\20200608\clip_49_2221 exists and overwrite is not set to true 
clip_4_0144
File E:\0_Monthly_videoPipeline\output_desat_clips\20200608\clip_4_0144 exists and overwrite is not set to true 
clip_50_2249
File E:\0_Monthly_videoPipeline\output_desat_clips\20200608\clip_50_2249 exists and overwrite is not set to true 
clip_51_2317
File E:\0_Monthly_videoPipeline\output_desat_clips\20200608\clip_51_2317 exists and overwrite is not set to true 
clip_52_2345
File E:\0_Monthly_videoPipeline\output_desat_clips\20200608\clip_52_2345 exists and overwrite is not set to true 
clip_5_0212
File E:\0_Monthly_videoPipeline\output_desat_clips\20200608\clip_5_0212 exists and overwrite is not set to true 
clip_6_0240
File E:\0_Monthly_videoPipeline\output_desat_clips\20200608\clip_6_0240 exists and overwrite is not set

clip_5_1538
File E:\0_Monthly_videoPipeline\output_desat_clips\20200624\clip_5_1538 exists and overwrite is not set to true 
clip_6_1606
File E:\0_Monthly_videoPipeline\output_desat_clips\20200624\clip_6_1606 exists and overwrite is not set to true 
clip_7_1634
File E:\0_Monthly_videoPipeline\output_desat_clips\20200624\clip_7_1634 exists and overwrite is not set to true 
clip_8_1702
File E:\0_Monthly_videoPipeline\output_desat_clips\20200624\clip_8_1702 exists and overwrite is not set to true 
clip_9_1725
File E:\0_Monthly_videoPipeline\output_desat_clips\20200624\clip_9_1725 exists and overwrite is not set to true 
clip_0_0000
File E:\0_Monthly_videoPipeline\output_desat_clips\20200625\clip_0_0000 exists and overwrite is not set to true 
clip_10_0440
File E:\0_Monthly_videoPipeline\output_desat_clips\20200625\clip_10_0440 exists and overwrite is not set to true 
clip_11_0508
File E:\0_Monthly_videoPipeline\output_desat_clips\20200625\clip_11_0508 exists and overwrite is not set to tr

...

clip_14_0632
About to run: ffmpeg -n -i E:\0_Monthly_videoPipeline\output_clips\20210427\clip_14_0632.mp4 -vf hue=s=0 E:\0_Monthly_videoPipeline\output_desat_clips\20210427\clip_14_0632.mp4
clip_15_0700
About to run: ffmpeg -n -i E:\0_Monthly_videoPipeline\output_clips\20210427\clip_15_0700.mp4 -vf hue=s=0 E:\0_Monthly_videoPipeline\output_desat_clips\20210427\clip_15_0700.mp4
clip_16_0724
About to run: ffmpeg -n -i E:\0_Monthly_videoPipeline\output_clips\20210427\clip_16_0724.mp4 -vf hue=s=0 E:\0_Monthly_videoPipeline\output_desat_clips\20210427\clip_16_0724.mp4
clip_17_0752
About to run: ffmpeg -n -i E:\0_Monthly_videoPipeline\output_clips\20210427\clip_17_0752.mp4 -vf hue=s=0 E:\0_Monthly_videoPipeline\output_desat_clips\20210427\clip_17_0752.mp4
clip_18_0820
About to run: ffmpeg -n -i E:\0_Monthly_videoPipeline\output_clips\20210427\clip_18_0820.mp4 -vf hue=s=0 E:\0_Monthly_videoPipeline\output_desat_clips\20210427\clip_18_0820.mp4
clip_19_0848
About to run: ffmpeg -n -i E:\0_Month

clip_5_0220
About to run: ffmpeg -n -i E:\0_Monthly_videoPipeline\output_clips\20210427\clip_5_0220.mp4 -vf hue=s=0 E:\0_Monthly_videoPipeline\output_desat_clips\20210427\clip_5_0220.mp4
clip_6_0248
About to run: ffmpeg -n -i E:\0_Monthly_videoPipeline\output_clips\20210427\clip_6_0248.mp4 -vf hue=s=0 E:\0_Monthly_videoPipeline\output_desat_clips\20210427\clip_6_0248.mp4
clip_7_0316
About to run: ffmpeg -n -i E:\0_Monthly_videoPipeline\output_clips\20210427\clip_7_0316.mp4 -vf hue=s=0 E:\0_Monthly_videoPipeline\output_desat_clips\20210427\clip_7_0316.mp4
clip_8_0344
About to run: ffmpeg -n -i E:\0_Monthly_videoPipeline\output_clips\20210427\clip_8_0344.mp4 -vf hue=s=0 E:\0_Monthly_videoPipeline\output_desat_clips\20210427\clip_8_0344.mp4
clip_9_0412
About to run: ffmpeg -n -i E:\0_Monthly_videoPipeline\output_clips\20210427\clip_9_0412.mp4 -vf hue=s=0 E:\0_Monthly_videoPipeline\output_desat_clips\20210427\clip_9_0412.mp4
clip_0_0000
About to run: ffmpeg -n -i E:\0_Monthly_videoPipeline

clip_45_2041
About to run: ffmpeg -n -i E:\0_Monthly_videoPipeline\output_clips\20210428\clip_45_2041.mp4 -vf hue=s=0 E:\0_Monthly_videoPipeline\output_desat_clips\20210428\clip_45_2041.mp4
clip_46_2109
About to run: ffmpeg -n -i E:\0_Monthly_videoPipeline\output_clips\20210428\clip_46_2109.mp4 -vf hue=s=0 E:\0_Monthly_videoPipeline\output_desat_clips\20210428\clip_46_2109.mp4
clip_47_2137
About to run: ffmpeg -n -i E:\0_Monthly_videoPipeline\output_clips\20210428\clip_47_2137.mp4 -vf hue=s=0 E:\0_Monthly_videoPipeline\output_desat_clips\20210428\clip_47_2137.mp4
clip_48_2205
About to run: ffmpeg -n -i E:\0_Monthly_videoPipeline\output_clips\20210428\clip_48_2205.mp4 -vf hue=s=0 E:\0_Monthly_videoPipeline\output_desat_clips\20210428\clip_48_2205.mp4
clip_49_2233
About to run: ffmpeg -n -i E:\0_Monthly_videoPipeline\output_clips\20210428\clip_49_2233.mp4 -vf hue=s=0 E:\0_Monthly_videoPipeline\output_desat_clips\20210428\clip_49_2233.mp4
clip_4_0152
About to run: ffmpeg -n -i E:\0_Monthl

clip_37_1634
About to run: ffmpeg -n -i E:\0_Monthly_videoPipeline\output_clips\20210429\clip_37_1634.mp4 -vf hue=s=0 E:\0_Monthly_videoPipeline\output_desat_clips\20210429\clip_37_1634.mp4
clip_38_1702
About to run: ffmpeg -n -i E:\0_Monthly_videoPipeline\output_clips\20210429\clip_38_1702.mp4 -vf hue=s=0 E:\0_Monthly_videoPipeline\output_desat_clips\20210429\clip_38_1702.mp4
clip_39_1730
About to run: ffmpeg -n -i E:\0_Monthly_videoPipeline\output_clips\20210429\clip_39_1730.mp4 -vf hue=s=0 E:\0_Monthly_videoPipeline\output_desat_clips\20210429\clip_39_1730.mp4
clip_3_0124
About to run: ffmpeg -n -i E:\0_Monthly_videoPipeline\output_clips\20210429\clip_3_0124.mp4 -vf hue=s=0 E:\0_Monthly_videoPipeline\output_desat_clips\20210429\clip_3_0124.mp4
clip_40_1758
About to run: ffmpeg -n -i E:\0_Monthly_videoPipeline\output_clips\20210429\clip_40_1758.mp4 -vf hue=s=0 E:\0_Monthly_videoPipeline\output_desat_clips\20210429\clip_40_1758.mp4
clip_41_1826
About to run: ffmpeg -n -i E:\0_Monthly_

clip_28_1226
About to run: ffmpeg -n -i E:\0_Monthly_videoPipeline\output_clips\20210430\clip_28_1226.mp4 -vf hue=s=0 E:\0_Monthly_videoPipeline\output_desat_clips\20210430\clip_28_1226.mp4
clip_29_1254
About to run: ffmpeg -n -i E:\0_Monthly_videoPipeline\output_clips\20210430\clip_29_1254.mp4 -vf hue=s=0 E:\0_Monthly_videoPipeline\output_desat_clips\20210430\clip_29_1254.mp4
clip_2_0056
About to run: ffmpeg -n -i E:\0_Monthly_videoPipeline\output_clips\20210430\clip_2_0056.mp4 -vf hue=s=0 E:\0_Monthly_videoPipeline\output_desat_clips\20210430\clip_2_0056.mp4
clip_30_1322
About to run: ffmpeg -n -i E:\0_Monthly_videoPipeline\output_clips\20210430\clip_30_1322.mp4 -vf hue=s=0 E:\0_Monthly_videoPipeline\output_desat_clips\20210430\clip_30_1322.mp4
clip_31_1350
About to run: ffmpeg -n -i E:\0_Monthly_videoPipeline\output_clips\20210430\clip_31_1350.mp4 -vf hue=s=0 E:\0_Monthly_videoPipeline\output_desat_clips\20210430\clip_31_1350.mp4
clip_32_1418
About to run: ffmpeg -n -i E:\0_Monthly_

# Initialize the weather gather class and create a metadata file containing weather information and timestamps for each clip

In [10]:
input_clips_dir = output_grayscaled_clips_dir
metadata_filename = "metadata.csv"
output_metadata_dir = os.path.join(input_clips_dir, metadata_filename) 


weatherGather = WeatherGather(pattern,input_clips_dir, output_metadata_dir, 'dmi')
weatherGather.createWeatherData()

['E:', '0_Monthly_videoPipeline', 'output_desat_clips', '20200514']
E:\0_Monthly_videoPipeline\output_desat_clips\20200514   2020-05-14 13:31:00    2020-05-14 23:35:00
<Response [200]> https://dmigw.govcloud.dk/metObs/v1/station?api-key=a017c060-41ce-4516-85ba-086bc77bed0a&limit=10000
<Response [200]> https://dmigw.govcloud.dk/metObs/v1/observation?api-key=a017c060-41ce-4516-85ba-086bc77bed0a&from=1589463060000000&to=1589499300000000&stationId=06031&limit=10000
['E:', '0_Monthly_videoPipeline', 'output_desat_clips', '20200515']
E:\0_Monthly_videoPipeline\output_desat_clips\20200515   2020-05-15 00:00:00    2020-05-15 23:57:00
<Response [200]> https://dmigw.govcloud.dk/metObs/v1/station?api-key=a017c060-41ce-4516-85ba-086bc77bed0a&limit=10000
<Response [200]> https://dmigw.govcloud.dk/metObs/v1/observation?api-key=a017c060-41ce-4516-85ba-086bc77bed0a&from=1589500800000000&to=1589587020000000&stationId=06031&limit=10000
['E:', '0_Monthly_videoPipeline', 'output_desat_clips', '20200516']


<Response [200]> https://dmigw.govcloud.dk/metObs/v1/observation?api-key=a017c060-41ce-4516-85ba-086bc77bed0a&from=1590883200000000&to=1590967920000000&stationId=06031&limit=10000
['E:', '0_Monthly_videoPipeline', 'output_desat_clips', '20200601']
E:\0_Monthly_videoPipeline\output_desat_clips\20200601   2020-06-01 00:00:00    2020-06-01 16:42:00
<Response [200]> https://dmigw.govcloud.dk/metObs/v1/station?api-key=a017c060-41ce-4516-85ba-086bc77bed0a&limit=10000
<Response [200]> https://dmigw.govcloud.dk/metObs/v1/observation?api-key=a017c060-41ce-4516-85ba-086bc77bed0a&from=1590969600000000&to=1591029720000000&stationId=06031&limit=10000
['E:', '0_Monthly_videoPipeline', 'output_desat_clips', '20200602']
E:\0_Monthly_videoPipeline\output_desat_clips\20200602   2020-06-02 11:59:00    2020-06-02 23:44:00
<Response [200]> https://dmigw.govcloud.dk/metObs/v1/station?api-key=a017c060-41ce-4516-85ba-086bc77bed0a&limit=10000
<Response [200]> https://dmigw.govcloud.dk/metObs/v1/observation?api

<Response [200]> https://dmigw.govcloud.dk/metObs/v1/station?api-key=a017c060-41ce-4516-85ba-086bc77bed0a&limit=10000
<Response [200]> https://dmigw.govcloud.dk/metObs/v1/observation?api-key=a017c060-41ce-4516-85ba-086bc77bed0a&from=1592438400000000&to=1592463600000000&stationId=06031&limit=10000
['E:', '0_Monthly_videoPipeline', 'output_desat_clips', '20200624']
E:\0_Monthly_videoPipeline\output_desat_clips\20200624   2020-06-24 13:18:00    2020-06-24 23:37:00
<Response [200]> https://dmigw.govcloud.dk/metObs/v1/station?api-key=a017c060-41ce-4516-85ba-086bc77bed0a&limit=10000
<Response [200]> https://dmigw.govcloud.dk/metObs/v1/observation?api-key=a017c060-41ce-4516-85ba-086bc77bed0a&from=1593004680000000&to=1593041820000000&stationId=06031&limit=10000
['E:', '0_Monthly_videoPipeline', 'output_desat_clips', '20200625']
E:\0_Monthly_videoPipeline\output_desat_clips\20200625   2020-06-25 00:00:00    2020-06-25 23:55:00
<Response [200]> https://dmigw.govcloud.dk/metObs/v1/station?api-key

<Response [200]> https://dmigw.govcloud.dk/metObs/v1/station?api-key=a017c060-41ce-4516-85ba-086bc77bed0a&limit=10000
<Response [200]> https://dmigw.govcloud.dk/metObs/v1/observation?api-key=a017c060-41ce-4516-85ba-086bc77bed0a&from=1597104000000000&to=1597190100000000&stationId=06031&limit=10000
['E:', '0_Monthly_videoPipeline', 'output_desat_clips', '20200812']
E:\0_Monthly_videoPipeline\output_desat_clips\20200812   2020-08-12 00:00:00    2020-08-12 23:50:00
<Response [200]> https://dmigw.govcloud.dk/metObs/v1/station?api-key=a017c060-41ce-4516-85ba-086bc77bed0a&limit=10000
<Response [200]> https://dmigw.govcloud.dk/metObs/v1/observation?api-key=a017c060-41ce-4516-85ba-086bc77bed0a&from=1597190400000000&to=1597276200000000&stationId=06031&limit=10000
['E:', '0_Monthly_videoPipeline', 'output_desat_clips', '20200813']
E:\0_Monthly_videoPipeline\output_desat_clips\20200813   2020-08-13 00:00:00    2020-08-13 23:48:00
<Response [200]> https://dmigw.govcloud.dk/metObs/v1/station?api-key

<Response [200]> https://dmigw.govcloud.dk/metObs/v1/station?api-key=a017c060-41ce-4516-85ba-086bc77bed0a&limit=10000
<Response [200]> https://dmigw.govcloud.dk/metObs/v1/observation?api-key=a017c060-41ce-4516-85ba-086bc77bed0a&from=1598659200000000&to=1598744040000000&stationId=06031&limit=10000
['E:', '0_Monthly_videoPipeline', 'output_desat_clips', '20200830']
E:\0_Monthly_videoPipeline\output_desat_clips\20200830   2020-08-30 00:00:00    2020-08-30 23:46:00
<Response [200]> https://dmigw.govcloud.dk/metObs/v1/station?api-key=a017c060-41ce-4516-85ba-086bc77bed0a&limit=10000
<Response [200]> https://dmigw.govcloud.dk/metObs/v1/observation?api-key=a017c060-41ce-4516-85ba-086bc77bed0a&from=1598745600000000&to=1598831160000000&stationId=06031&limit=10000
['E:', '0_Monthly_videoPipeline', 'output_desat_clips', '20200831']
E:\0_Monthly_videoPipeline\output_desat_clips\20200831   2020-08-31 00:00:00    2020-08-31 23:48:00
<Response [200]> https://dmigw.govcloud.dk/metObs/v1/station?api-key

...

<Response [200]> https://dmigw.govcloud.dk/metObs/v1/station?api-key=a017c060-41ce-4516-85ba-086bc77bed0a&limit=10000
<Response [200]> https://dmigw.govcloud.dk/metObs/v1/observation?api-key=a017c060-41ce-4516-85ba-086bc77bed0a&from=1612828800000000&to=1612914600000000&stationId=06031&limit=10000
['E:', '0_Monthly_videoPipeline', 'output_desat_clips', '20210210']
E:\0_Monthly_videoPipeline\output_desat_clips\20210210   2021-02-10 00:00:00    2021-02-10 23:39:00
<Response [200]> https://dmigw.govcloud.dk/metObs/v1/station?api-key=a017c060-41ce-4516-85ba-086bc77bed0a&limit=10000
<Response [200]> https://dmigw.govcloud.dk/metObs/v1/observation?api-key=a017c060-41ce-4516-85ba-086bc77bed0a&from=1612915200000000&to=1613000340000000&stationId=06031&limit=10000
['E:', '0_Monthly_videoPipeline', 'output_desat_clips', '20210211']
E:\0_Monthly_videoPipeline\output_desat_clips\20210211   2021-02-11 00:00:00    2021-02-11 18:44:00
<Response [200]> https://dmigw.govcloud.dk/metObs/v1/station?api-key

<Response [200]> https://dmigw.govcloud.dk/metObs/v1/station?api-key=a017c060-41ce-4516-85ba-086bc77bed0a&limit=10000
<Response [200]> https://dmigw.govcloud.dk/metObs/v1/observation?api-key=a017c060-41ce-4516-85ba-086bc77bed0a&from=1614432240000000&to=1614468720000000&stationId=06031&limit=10000
['E:', '0_Monthly_videoPipeline', 'output_desat_clips', '20210228']
E:\0_Monthly_videoPipeline\output_desat_clips\20210228   2021-02-28 00:00:00    2021-02-28 23:47:00
<Response [200]> https://dmigw.govcloud.dk/metObs/v1/station?api-key=a017c060-41ce-4516-85ba-086bc77bed0a&limit=10000
<Response [200]> https://dmigw.govcloud.dk/metObs/v1/observation?api-key=a017c060-41ce-4516-85ba-086bc77bed0a&from=1614470400000000&to=1614556020000000&stationId=06031&limit=10000
['E:', '0_Monthly_videoPipeline', 'output_desat_clips', '20210301']
E:\0_Monthly_videoPipeline\output_desat_clips\20210301   2021-03-01 00:00:00    2021-03-01 23:37:00
<Response [200]> https://dmigw.govcloud.dk/metObs/v1/station?api-key

<Response [200]> https://dmigw.govcloud.dk/metObs/v1/station?api-key=a017c060-41ce-4516-85ba-086bc77bed0a&limit=10000
<Response [200]> https://dmigw.govcloud.dk/metObs/v1/observation?api-key=a017c060-41ce-4516-85ba-086bc77bed0a&from=1615939200000000&to=1616023980000000&stationId=06031&limit=10000
['E:', '0_Monthly_videoPipeline', 'output_desat_clips', '20210318']
E:\0_Monthly_videoPipeline\output_desat_clips\20210318   2021-03-18 00:00:00    2021-03-18 23:58:00
<Response [200]> https://dmigw.govcloud.dk/metObs/v1/station?api-key=a017c060-41ce-4516-85ba-086bc77bed0a&limit=10000
<Response [200]> https://dmigw.govcloud.dk/metObs/v1/observation?api-key=a017c060-41ce-4516-85ba-086bc77bed0a&from=1616025600000000&to=1616111880000000&stationId=06031&limit=10000
['E:', '0_Monthly_videoPipeline', 'output_desat_clips', '20210319']
E:\0_Monthly_videoPipeline\output_desat_clips\20210319   2021-03-19 00:00:00    2021-03-19 23:41:00
<Response [200]> https://dmigw.govcloud.dk/metObs/v1/station?api-key

<Response [200]> https://dmigw.govcloud.dk/metObs/v1/station?api-key=a017c060-41ce-4516-85ba-086bc77bed0a&limit=10000
<Response [200]> https://dmigw.govcloud.dk/metObs/v1/observation?api-key=a017c060-41ce-4516-85ba-086bc77bed0a&from=1617494400000000&to=1617580260000000&stationId=06031&limit=10000
['E:', '0_Monthly_videoPipeline', 'output_desat_clips', '20210405']
E:\0_Monthly_videoPipeline\output_desat_clips\20210405   2021-04-05 00:00:00    2021-04-05 23:42:00
<Response [200]> https://dmigw.govcloud.dk/metObs/v1/station?api-key=a017c060-41ce-4516-85ba-086bc77bed0a&limit=10000
<Response [200]> https://dmigw.govcloud.dk/metObs/v1/observation?api-key=a017c060-41ce-4516-85ba-086bc77bed0a&from=1617580800000000&to=1617666120000000&stationId=06031&limit=10000
['E:', '0_Monthly_videoPipeline', 'output_desat_clips', '20210406']
E:\0_Monthly_videoPipeline\output_desat_clips\20210406   2021-04-06 00:00:00    2021-04-06 23:38:00
<Response [200]> https://dmigw.govcloud.dk/metObs/v1/station?api-key

<Response [200]> https://dmigw.govcloud.dk/metObs/v1/station?api-key=a017c060-41ce-4516-85ba-086bc77bed0a&limit=10000
<Response [200]> https://dmigw.govcloud.dk/metObs/v1/observation?api-key=a017c060-41ce-4516-85ba-086bc77bed0a&from=1619049600000000&to=1619135280000000&stationId=06031&limit=10000
['E:', '0_Monthly_videoPipeline', 'output_desat_clips', '20210423']
E:\0_Monthly_videoPipeline\output_desat_clips\20210423   2021-04-23 00:00:00    2021-04-23 23:36:00
<Response [200]> https://dmigw.govcloud.dk/metObs/v1/station?api-key=a017c060-41ce-4516-85ba-086bc77bed0a&limit=10000
<Response [200]> https://dmigw.govcloud.dk/metObs/v1/observation?api-key=a017c060-41ce-4516-85ba-086bc77bed0a&from=1619136000000000&to=1619220960000000&stationId=06031&limit=10000
['E:', '0_Monthly_videoPipeline', 'output_desat_clips', '20210424']
E:\0_Monthly_videoPipeline\output_desat_clips\20210424   2021-04-24 00:00:00    2021-04-24 23:33:00
<Response [200]> https://dmigw.govcloud.dk/metObs/v1/station?api-key